In [1]:
%pip install -q paddlepaddle
%pip install -q paddleocr
%pip install pandas tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from paddleocr import PaddleOCR
from PIL import Image
from utils import download_images
import shutil
from multiprocessing import Pool
from tqdm import tqdm
import sys
import gc
from IPython.display import clear_output

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
DATASET_FOLDER = "./"
IMAGE_FOLDER = "./images"
TEST_IMAGES = os.path.join(IMAGE_FOLDER, "test")
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
test["output"] = ""

In [4]:
import cv2

In [5]:
def find_text(image_link, path, ocr):
    image = image_link.split("/")[-1]
    image_path = os.path.join(path, image)
    if not os.path.exists(image_path):
        return ""
    image = cv2.imread(image_path)
    if image is None:
        # If image fails to load, return empty string
        print(f"Failed to load image at {image_path}")
        return ""
    try:
      result = ocr.ocr(image_path, cls=True)[0]
    except:
       text = ""
    try:
        text = '\n'.join([line[1][0] for line in result])
    except:
        text = ""
    return text

def process_data(train, start, end, path=TEST_IMAGES):
  ocr = PaddleOCR(use_angle_cls=True, lang='en',use_gpu=False)
  download_images(train[start:end]["image_link"], path)
  for index, row in tqdm(train[start:end].iterrows()):
    result = find_text(row['image_link'], path, ocr)
    if result != "":
      train.at[index, 'output'] = result
  del ocr
  gc.collect()
  return train

def restart():
  if os.path.exists(TEST_IMAGES):
    shutil.rmtree(TEST_IMAGES)
  os.makedirs(TEST_IMAGES)

In [6]:
start = 104500
end = 105000

In [8]:
while end <= 110000:
  print("\n[INFORMATION]", " Processing Upto: ", end,"\n")
  test = process_data(test, start, end, TEST_IMAGES)
  print("\n[INFORMATION]", " Creating checkpoint\n")
  test.to_csv(f'./results/test_output_{end}.csv', index=False)
  restart()
  start = end
  end = min(end + 500, 110000)
  clear_output(wait=True)
  # with Pool() as pool:
  #   for index, result in tqdm(pool.imap(process_row, train.iterrows()), total=len(train["image_link"])):
  #     if train.at[index, "output"] == "" and result != "":
  #       train.at[index, 'output'] = result
  #       count += 1
  #     if count == end - start:
  #       break
  # train.to_csv('/content/drive/MyDrive/train_output.csv', index=False)
  # if os.path.exists(TRAIN_IMAGES):
  #   shutil.rmtree(TRAIN_IMAGES)
  #   os.makedirs(TRAIN_IMAGES)


[INFORMATION]  Processing Upto:  106000 

[2024/09/16 04:21:36] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/rishisharma/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/rishisharma/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_i

 24%|██▍       | 119/500 [05:51<18:46,  2.96s/it]




--------------------------------------
C++ Traceback (most recent call last):
--------------------------------------
No stack trace in paddle, may be caused by external reasons.

----------------------
Error Message Summary:
----------------------
FatalError: `Termination signal` is detected by the operating system.
  [TimeInfo: *** Aborted at 1726441056 (unix time) try "date -d @1726441056" if you are using GNU date ***]
  [SignalInfo: *** SIGTERM (@0x3ed0000c21f) received by PID 50369 (TID 0x7cd3595c31c0) from PID 49695 ***]



--------------------------------------
C++ Traceback (most recent call last):
--------------------------------------
No stack trace in paddle, may be caused by external reasons.

----------------------
Error Message Summary:
----------------------
FatalError: `Termination signal` is detected by the operating system.
  [TimeInfo: *** Aborted at 1726441056 (unix time) try "date -d @1726441056" if you are using GNU date ***]
  [SignalInfo: *** SIGTERM (@0x3ed00

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv('results/test_output_20000.csv')

In [ ]:
df["output"][19999]

'3.3 in/8.5 cm\n8.3 in/21cm'